# Lagrangian Mechanics: constrained movement

In [ ]:
import sympy, scipy, matplotlib

from mechanics import *

## Quadratic constraint

In [ ]:
g = symbols('g',positive=True)

Q = [x] = dynsyms(['x'])

n = 1
dim = 2
P,V = posvel(n,dim)
M   = vector(n,'m')
T   = M.dot( mul(V,V) * ones(dim,1) ) / 2
V   = g * M.dot(P[:,-1])

Gen = mkGen(P, [x,
                x**2] )


sys = Dynamics(T, V, Gen, Q, {})

In [ ]:
disp(sys.Q, 'Q')

In [ ]:
[ Eq(u,v) for u,v in Gen.items() ]

In [ ]:
disp(T,'T')

In [ ]:
disp(sys.T.simplify(),'T')

In [ ]:
disp(V,'V')

In [ ]:
disp(sys.V.simplify(),'V')

In [ ]:
disp(sys.L.simplify(),'L')

In [ ]:
[ Eq(v,solve(e,v)[0]) for e,v in zip(sys.ecsL,sys.D2) ]

In [ ]:
case = {g:1, **val(M,[1])}
sys = Dynamics(T, V, Gen, Q, case)
fps = 30

In [ ]:
sol = nsolve(sys.dotL, T=10, dt=1/fps, q0=[1.5, 0])

graph(sol,sys)

In [ ]:
def prepare():
    fig = plt.figure(figsize=(4,4))
    ax = plt.gca()
    plt.tight_layout()
    ax.axis('equal')
    plt.close();

    x = np.linspace(-2,2,100)
    V = x**2

    ax.fill_between(x,-0.1,V,color='lightgray')
    line0, = ax.plot([], [], '.-', lw=2, markersize=20,color='#1f77b4')

    def draw(k, t, x1,y1):
        line0.set_data([x1],[y1])

    return fig, draw

In [ ]:
ani = mkAnim(sol, sys, prepare, fps, frames=len(sol))
HTML(ani.to_jshtml())

Para oscilaciones pequeñas se va pareciendo cada vez más a un oscilador armónico:

In [ ]:
sol = nsolve(sys.dotL, T=10, dt=1/fps, q0=[0.5, 0])
graph(sol,sys)

In [ ]:
sol = nsolve(sys.dotL, T=10, dt=1/fps, q0=[0.1, 0])
graph(sol,sys)

## Brachistochrone

Verificamos que la trayectoria más rápida es la cicloide comparando con algunas alternativas.

Los parámetros de la curva dependen de la distancia y desnivel.

https://scipython.com/blog/the-brachistochrone-problem/

In [ ]:
def brach_param(dx,dy,g):
    from scipy.optimize import newton

    def f(theta):
        return dy/dx - (1-np.cos(theta))/(theta-np.sin(theta))
    theta2 = newton(f, np.pi/2)

    r = 1 / (1 - np.cos(theta2))

    pi = np.pi
    t = np.linspace(0,theta2)

    T = theta2 * np.sqrt(r / g)

    x = r*(t-np.sin(t))
    y = -r*(1-np.cos(t))

    plt.figure(figsize=(4,4))
    plt.plot(x,y)

    plt.axis('equal');
    plt.grid();
    return r, theta2, T

In [ ]:
dx,dy, my_g = 3,1, 1
my_r, my_theta, travel_time = brach_param(dx,dy,my_g)

In [ ]:
g, r, X,Y = symbols('g r X Y',positive=True)

Q = [a,b,c] = dynsyms(['a', 'b', 'c'])

n = 3
dim = 2
P,V = posvel(n,dim)
M   = vector(n,'m')
T   = M.dot( mul(V,V) * ones(dim,1) ) / 2
V   = g * M.dot(P[:,-1])

Gen = mkGen(P, [a,-a*Y/X,
                b,(b-X)**2/X**2-Y,
                r*(c-sin(c)), -r*(1-cos(c))
               ]
           )

sys = Dynamics(T, V, Gen, Q, {})

In [ ]:
disp(sys.Q, 'Q')

In [ ]:
[ Eq(u,v) for u,v in Gen.items() ]

In [ ]:
disp(T,'T')

In [ ]:
disp(sys.T.simplify(),'T')

In [ ]:
disp(V,'V')

In [ ]:
disp(sys.V.simplify(),'V')

In [ ]:
disp(sys.L.simplify(),'L')

In [ ]:
[ Eq(v,solve(e,v)[0]) for e,v in zip(sys.ecsL,sys.D2) ]

In [ ]:
case = {g:my_g, r:my_r, X:dx, Y:dy, **val(M,[1,1,1])}
sys = Dynamics(T, V, Gen, Q, case)
fps = 30

In [ ]:
sol = nsolve(sys.dotL, T=travel_time, dt=1/fps, q0=[0, 0, 0.00001, 0, 0, 0])

In [ ]:
graph(sol,sys)

In [ ]:
def prepare():
    fig = plt.figure(figsize=(4,4))
    ax = plt.gca()
    plt.tight_layout()
    ax.axis('equal')
    plt.close();

    x = np.linspace(0,dx,100)
    V = x**2
    ax.plot(x,-x*dy/dx)
    ax.plot(x,(x-dx)**2/dx**2-dy)
    t = np.linspace(0,my_theta)
    r = my_r

    ax.plot( r*(t-np.sin(t)) ,-r*(1-np.cos(t)));
    line0, = ax.plot([], [], '.-', lw=2, markersize=20,color='#1f77b4',label='linear')
    line1, = ax.plot([], [], '.-', lw=2, markersize=20,color='orange',label='quadratic')
    line2, = ax.plot([], [], '.-', lw=2, markersize=20,color='green',label='cycloid')
    ax.legend()

    def draw(k, t, x1,y1,x2,y2,x3,y3):
        line0.set_data([x1],[y1])
        line1.set_data([x2],[y2])
        line2.set_data([x3],[y3])

    return fig, draw

In [ ]:
ani = mkAnim(sol, sys, prepare, fps, frames=len(sol))
HTML(ani.to_jshtml(default_mode='once'))

Es curioso que en este caso tenga un tramo de subida.

Otra configuración:

In [ ]:
dx,dy, my_g = 1,1,1
my_r, my_theta, travel_time = brach_param(dx,dy,my_g)

In [ ]:
case = {g:my_g, r:my_r, X:dx, Y:dy, **val(M,[1,1,1])}
sys = Dynamics(T, V, Gen, Q, case)
fps = 30

In [ ]:
sol = nsolve(sys.dotL, T=travel_time, dt=1/fps, q0=[0, 0, 0.00001, 0, 0, 0])

In [ ]:
ani = mkAnim(sol, sys, prepare, fps, frames=len(sol))
HTML(ani.to_jshtml(default_mode='once'))